In [1]:
import scipy
import pandas as pd
import numpy as np
import statsmodels.api as sm

In [2]:
df = pd.read_csv("Data.csv", sep=";")
df.dtypes

Direksi Perempuan    int64
Direksi Laki-Laki    int64
Harga penutupan      int64
Total Shares         int64
Total Utang          int64
Total Asset          int64
dtype: object

In [3]:
#X = (df['Direksi Perempuan'])/(df['Direksi Perempuan'] + df['Direksi Laki-Laki'])
X = df["Direksi Perempuan"]
nilai_pasar = df["Total Shares"] * df["Harga penutupan"]
np_plus_liability = nilai_pasar + df["Total Utang"]
Y = np_plus_liability / df["Total Asset"]
#Y = nilai_pasar / df["Total Asset"]

data = pd.DataFrame({"X":X, "Y":Y})
X_mean = np.mean(data["X"])
X_std = np.std(data["X"])
Y_mean = np.mean(data["Y"])
Y_std = np.std(data["Y"])

In [4]:
data_sample = pd.DataFrame({"X": X, "Y": Y})
data_sample.describe()

,X,Y
count,140.000000,140.000000
mean,1.300000,1.311243
std,1.239053,2.089952
min,0.000000,0.159993
25%,0.000000,0.928060
50%,1.000000,0.974360
75%,2.000000,1.101783
max,6.000000,18.327231


In [5]:
# data_sample.drop(data_sample[data_sample["X"] == 0].index, inplace=True)
# np.size(np.where(data_sample["X"] == 0))

In [6]:
X_with_constant = sm.add_constant(data_sample["X"])
result = sm.OLS(data_sample["Y"], X_with_constant).fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      Y   R-squared:                       0.003
Model:                            OLS   Adj. R-squared:                 -0.004
Method:                 Least Squares   F-statistic:                    0.4012
Date:                Sun, 07 May 2023   Prob (F-statistic):              0.528
Time:                        23:40:17   Log-Likelihood:                -301.15
No. Observations:                 140   AIC:                             606.3
Df Residuals:                     138   BIC:                             612.2
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.4293      0.257      5.560      0.000       0.921       1.938
X             -0.0908      0.143     -0.633      0.528      -0.374       0.193
==============================================================================
Omnibus:                      238.353   Durbin-Watson:                   0.889
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            22155.498
Skew:                           7.700   Prob(JB):                         0.00
Kurtosis:                      62.673   Cond. No.                         3.09
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [7]:
bX = data_sample["X"] * result.params[0]
abs_residual = np.abs(data_sample["Y"] - result.params["const"] - bX)

In [8]:
hetero_result = sm.OLS(abs_residual, X_with_constant).fit()
hetero_result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.516
Model:                            OLS   Adj. R-squared:                  0.513
Method:                 Least Squares   F-statistic:                     147.4
Date:                Sun, 07 May 2023   Prob (F-statistic):           1.57e-23
Time:                        23:40:18   Log-Likelihood:                -266.86
No. Observations:                 140   AIC:                             537.7
Df Residuals:                     138   BIC:                             543.6
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.7327      0.201      3.641      0.000       0.335       1.131
X              1.3627      0.112     12.141      0.000       1.141       1.585
==============================================================================
Omnibus:                      240.021   Durbin-Watson:                   1.133
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            22991.073
Skew:                           7.789   Prob(JB):                         0.00
Kurtosis:                      63.817   Cond. No.                         3.09
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [9]:
XQ3,XQ1 = np.percentile(data_sample["X"], [75 ,25])
XIQR = XQ3 - XQ1 

In [10]:
lower_X = XQ1 - (1.5 * XIQR)
upper_X = XQ3 + (1.5 * XIQR)

In [11]:
data_sample["X"][(data_sample["X"] < lower_X) | (data_sample["X"] > upper_X)]

70    6
71    6
72    6
Name: X, dtype: int64

In [12]:
data_sample.drop(data_sample["X"][(data_sample["X"] < lower_X) | (data_sample["X"] > upper_X)].index, inplace=True)

In [13]:
YQ3,YQ1 = np.percentile(data_sample['Y'], [75 ,25])
YIQR = YQ3 - YQ1 

In [14]:
lower_Y = YQ1 - (1.5 * YIQR)
upper_Y = YQ3 + (1.5 * YIQR)

In [15]:
data_sample.drop(data_sample["Y"][(data_sample["Y"] < lower_Y) | (data_sample["Y"] > upper_Y)].index, inplace=True)

In [16]:
data_sample.describe()

,X,Y
count,114.000000,114.000000
mean,1.342105,1.003624
std,1.037602,0.119201
min,0.000000,0.688744
25%,1.000000,0.933108
50%,1.000000,0.970694
75%,2.000000,1.057050
max,4.000000,1.385464


In [17]:
X_with_constant = sm.add_constant(data_sample["X"])
result = sm.OLS(data_sample["Y"], X_with_constant).fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      Y   R-squared:                       0.013
Model:                            OLS   Adj. R-squared:                  0.004
Method:                 Least Squares   F-statistic:                     1.494
Date:                Sun, 07 May 2023   Prob (F-statistic):              0.224
Time:                        23:40:18   Log-Likelihood:                 81.970
No. Observations:                 114   AIC:                            -159.9
Df Residuals:                     112   BIC:                            -154.5
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.9859      0.018     53.984      0.000       0.950       1.022
X              0.0132      0.011      1.222      0.224      -0.008       0.035
==============================================================================
Omnibus:                       18.919   Durbin-Watson:                   1.046
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               22.433
Skew:                           0.990   Prob(JB):                     1.35e-05
Kurtosis:                       3.897   Cond. No.                         3.46
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [18]:
bX = data_sample["X"] * result.params["X"]

In [19]:
# Y = a + bX + e
# e = Y - a - bX

abs_residual = np.abs(data_sample["Y"] - result.params["const"] - bX)

In [20]:
hetero_result = sm.OLS(abs_residual, X_with_constant).fit()
hetero_result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.009
Method:                 Least Squares   F-statistic:                  0.005176
Date:                Sun, 07 May 2023   Prob (F-statistic):              0.943
Time:                        23:40:18   Log-Likelihood:                 135.52
No. Observations:                 114   AIC:                            -267.0
Df Residuals:                     112   BIC:                            -261.6
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0914      0.011      8.002      0.000       0.069       0.114
X              0.0005      0.007      0.072      0.943      -0.013       0.014
==============================================================================
Omnibus:                       42.939   Durbin-Watson:                   1.327
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               87.357
Skew:                           1.578   Prob(JB):                     1.07e-19
Kurtosis:                       5.903   Cond. No.                         3.46
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""